In [1]:
from dn3.configuratron import ExperimentConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet

import mne
mne.set_log_level(False)

In [10]:
config_filename = 'my_config.yml'
experiment = ExperimentConfig(config_filename)
ds_config = experiment.datasets['mmidb']

dataset = ds_config.auto_construct_dataset()

Scanning /home/chris/Desktop/DL/mnist eeg/BENDR/eeg-motor-movementimagery-dataset-1.0.0. If there are a lot of files, this may take a while...: 100%|██████████| 4/4 [00:00<00:00, 77.21it/s, extension=.gdf]
Loading Physionet MMIDB:   0%|          | 0/105 [00:00<?, ?person/s]

Adding additional configuration entries: dict_keys(['train_params', 'lr', 'folds'])
Configuratron found 1 datasets.
Creating dataset of 315 Preloaded Epoched recordings from 105 people.


Loading Physionet MMIDB: 100%|██████████| 105/105 [00:11<00:00,  8.96person/s]


>> Physionet MMIDB | DSID: None | 105 people | 4408 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Constructed 1 channel maps
Used by 315 recordings:
EEG (original(new)): Fc5.(FC5) Fc3.(FC3) Fc1.(FC1) Fcz.(FCZ) Fc2.(FC2) Fc4.(FC4) Fc6.(FC6) C5..(C5) C3..(C3) C1..(C1) Cz..(CZ) C2..(C2) C4..(C4) C6..(C6) Cp5.(CP5) Cp3.(CP3) Cp1.(CP1) Cpz.(CPZ) Cp2.(CP2) Cp4.(CP4) Cp6.(CP6) Fp1.(FP1) Fpz.(FPZ) Fp2.(FP2) Af7.(AF7) Af3.(AF3) Afz.(AFZ) Af4.(AF4) Af8.(AF8) F7..(F7) F5..(F5) F3..(F3) F1..(F1) Fz..(FZ) F2..(F2) F4..(F4) F6..(F6) F8..(F8) Ft7.(FT7) Ft8.(FT8) T7..(T7) T8..(T8) T9..(T9) T10.(T10) Tp7.(TP7) Tp8.(TP8) P7..(P7) P5..(P5) P3..(P3) P1..(P1) Pz..(PZ) P2..(P2) P4..(P4) P6..(P6) P8..(P8) Po7.(PO7) Po3.(PO3) Poz.(POZ) Po4.(PO4) Po8.(PO8) O1..(O1) Oz..(OZ) O2..(O2) Iz..(IZ) 
EOG (original(new)): 
REF (original(new)): 
EXTRA (original(new)): 
Heuristically Assigned: Fc5.(FC5)  Fc3.(FC3)  Fc1.(FC1)  Fcz.(FCZ)  Fc2.(FC2)  Fc4.(FC4)  Fc6.(FC6)  C5..(C5)  C3..(C3)  C1..(C1)  Cz

In [22]:
dataset.lmso()

<generator object Dataset.lmso at 0x7f123c9a3040>

In [36]:
experiment.datasets['mmidb'].train_params.epochs

7

In [28]:
experiment.

TypeError: 'ExperimentConfig' object is not subscriptable

In [11]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet, cuda=experiment.use_gpu, learning_rate=ds_config.lr)

In [29]:
results = dict()
for training, validation, test in dataset.lmso():
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation,
                epochs=experiment.datasets['mmidb'].train_params.epochs,
                batch_size=experiment.datasets['mmidb'].train_params.batch_size)

    for _, _, test_thinker in test.loso():
        results[test_thinker.person_id] = process.evaluate(test_thinker)

print(results)
print("Average accuracy: {:.2%}".format(sum(results.values())/len(results)))

Training:   >> Physionet MMIDB | DSID: None | 84 people | 3528 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Validation: >> Physionet MMIDB | DSID: None | 10 people | 418 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Test:       >> Physionet MMIDB | DSID: None | 11 people | 462 trials | 90 channels | 960 samples/trial | 160.0Hz | 0 transforms
Creating TIDNet using: 90 channels x 960 samples at 160.0Hz | 2 targets


TypeError: 'DatasetConfig' object is not subscriptable